### Instructions

1. Plug in the `model_path` (and change `script_path` and `singularity_path` if neccessary).
2. From `Run` menu, select `Restart Kernel and Run All Cells`
3. From `View` menu, select `Collapse All Code`

In [58]:
model_path = '/data00/projects/megameta/scripts/jupyter_megameta/first_level_models/BA/model_specifications/task-walkstatement_model-messageUpdateTest.json'
            

In [59]:
script_path = '/data00/projects/megameta/scripts/jupyter_megameta/cnlab_pipeline/cnlab/GLM/first_level_HY.py'
singularity_path = '/data00/tools/singularity_images/neurodocker.sqsh'

In [60]:
import os, glob, json, copy
import pandas as pd
import numpy as np

In [61]:
def ensure_exist(path, file):
    if os.path.exists(os.path.join(path, file)):
        return file
    else:
        raise Exception("File missing: " + file)

def ensure_relative(path):
    if path.startswith('/'):
        raise Exception("Make sure path is relative to the data path: " + path)
    else:
        return path
        
def ensure_list(obj):
    if type(obj) is not list:
        return [obj]
    else:
        return obj
    
def remove_key(key, obj):
    if obj.get(key):
        del obj[key]
        
def copy_from_template(target, template):
    for key, item in template.items():
        if key not in target.keys():
            target[key] = copy.deepcopy(template[key])
        elif type(item) is dict:
            copy_from_template(target[key], template[key])

In [69]:
with open(model_path, 'r') as f:
    model = json.load(f)
    
if model.get("Template"):

    template_path = model.get("Template")
    with open(template_path, 'r') as f:
        template = json.load(f)
    
    print(f"Using template {os.path.basename(template_path)}: ", end="")
    print("\n\t".join(template.get("Descriptions", ["No description found"])))

    copy_from_template(model, template)

Using template megameta_default.json: Mega-meta default pipeline
	6mm FWHM smoothing
	No global scaling
	FAST correlation
	6 motion parameters and framewise displacement
	Outlier: FD > 0.75


In [70]:
task = model['Info']['task']
job_name = 'task-{}_model-{}'.format(model['Info']['task'], model['Info']['model'])

env = model['Environment']

os.makedirs(env['output_path'], exist_ok=True)
os.makedirs(env['working_path'], exist_ok=True)

try:
    os.chmod(env['output_path'], 0o0777)
    os.chmod(env['working_path'], 0o0777)
except:
    pass
    
subs = []
if model['Info'].get('sub'):
    subs = ensure_list(model['Info']['sub'])
elif model['Info'].get('sub_container'):
    sub_container = ensure_relative(model['Info']['sub_container'])
    subs = [x.split(os.path.sep)[-2].replace('sub-','') for x in glob.glob(os.path.join(env['data_path'], sub_container, 'sub-*'+os.path.sep))]

exclude_subs = model['Info'].get('exclude',{}).get('sub',[])
for es in exclude_subs:
    if es in subs:
        subs.remove(es)

if len(subs) == 0:
    raise Exception("No subjects found.")
else:
    print(f"Processing {len(subs)} participants:")
    print(subs)

Processing 34 participants:
['BA224', 'BA227', 'BA228', 'BA229', 'BA234', 'BA235', 'BA237', 'BA238', 'BA239', 'BA240', 'BA242', 'BA244', 'BA247', 'BA250', 'BA251', 'BA252', 'BA253', 'BA255', 'BA257', 'BA258', 'BA261', 'BA262', 'BA263', 'BA265', 'BA266', 'BA267', 'BA269', 'BA272', 'BA273', 'BA275', 'BA276', 'BA277', 'BA278', 'BA245']


In [71]:
def customize_event(env, job_name, spm_model, event_path):
    
    output_path = os.path.join(ensure_relative(env['job_path']), job_name, 'events') 
    event_df = pd.read_csv(os.path.join(env['data_path'], event_path), sep='\t')
    
    for operation, params in spm_model.get("event_options", {}).items():
        
        if operation == 'map_condition':
            
            condition_map = {}
            for trial_type in event_df['trial_type'].unique():
                condition_map[trial_type] = trial_type
                
            for new_trial_type, trial_types in params.items():
                for trial_type in ensure_list(trial_types):
                    condition_map[trial_type] = new_trial_type
            
            event_df['trial_type'] = event_df['trial_type'].map(condition_map)
            
        elif operation == 'melt_condition':
            
            for trial_type, trial_data in params.items():
                for idx, row in event_df.query(f'trial_type == "{trial_type}"').iterrows():
                    event_df.loc[idx, 'trial_type'] = trial_type + "_" + str(row[trial_data])

    all_pmods = []
    for trial_type, pmods in spm_model.get('pmod',{}).items():
        
        trial_df = event_df.query(f'trial_type == "{trial_type}"')
                
        for pmod in ensure_list(pmods):
            all_pmods.append(pmod)
            
            pmod_values = trial_df[pmod].copy()
            pmod_values = pmod_values.fillna(pmod_values.mean())
            
            for operation in ensure_list(spm_model.get("pmod_options", [])):
                
                if operation == "rank":
                    pmod_values = pmod_values.rank()
                elif operation == "minmax_scale":
                    pmod_values = (pmod_values - pmod_values.min()) / (pmod_values.max() - pmod_values.min())
                elif operation == "zscore":
                    pmod_values = (pmod_values - pmod_values.mean()) / pmod_values.std()
            
            event_df.loc[trial_df.index, pmod] = pmod_values.tolist()
        
    (event_df[['onset', 'duration', 'trial_type'] + np.unique(all_pmods).tolist()]
         .sort_values(by='onset')
         .to_csv(os.path.join(env['data_path'], output_path, os.path.basename(event_path)), sep='\t', index=False))
    
    return os.path.join(output_path, os.path.basename(event_path))

In [72]:
spm_model = model['SpecifySPMModel']

env['job_path'] = ensure_relative(env['job_path'])

job_path = os.path.join(env['data_path'], env['job_path'], job_name, 'jobs') 
os.makedirs(job_path, exist_ok=True)

if (spm_model.get("event_options") is not None) or (spm_model.get("pmod_options") is not None):
    event_path = os.path.join(env['job_path'], job_name, 'events') 
    os.makedirs(os.path.join(env['data_path'], event_path), exist_ok=True)

if spm_model.get("outlier"):
    outlier_path = os.path.join(env['job_path'], job_name, 'outlier') 
    os.makedirs(os.path.join(env['data_path'], outlier_path), exist_ok=True)
    
for regressors_path in ensure_list(spm_model.get('regressors', [])):
    if (not regressors_path.endswith('txt')) and (not regressors_path.endswith('tsv')):
        raise Exception("Regressors: only TSV or TXT file supported.")

regressor_names = ensure_list(spm_model.get('regressor_names',[[]]))
if type(regressor_names[0]) is not list:
    regressor_names = [regressor_names]
    
for sub in subs:
    print(sub, ": ", end="")
    
    job = copy.deepcopy(model)
    
    format_args = {}
    format_args['sub'] = sub
    format_args['task'] = task
    
    issues = []
    
    runs = ensure_list(job['Info'].get('run',-1))
    for exclude_sub, exclude_runs in job['Info'].get('exclude',{}).get('run',{}).items():
        if sub == exclude_sub:
            for exclude_run in ensure_list(exclude_runs):
                if exclude_run in runs:
                    runs.remove(exclude_run)
                    
    functional_runs = []
    regressors = []
    event_files = []
    outlier_files = []

    for run in runs:
        
        if run != -1:
            format_args['run'] = run
        
        functional_runs.append(ensure_exist(env['data_path'], spm_model['functional_runs'].format(**format_args)))
            
        regressors_path = ensure_exist(env['data_path'], spm_model['regressors'].format(**format_args))
        regressors.append(regressors_path)

        if regressors_path.endswith('tsv'):
            
            regressor_df = pd.read_csv(os.path.join(env['data_path'], regressors_path), sep='\t')

            chosen_regressor_names = []
            for rn in regressor_names:
                col_exists = regressor_df.columns.isin(rn)
                if col_exists.sum() > 0:
                    chosen_regressor_names = regressor_df.columns[col_exists].tolist()
                    if col_exists.sum() != len(rn):
                        issues.append(f"{regressors_path}: Only some of the regressors are found - {chosen_regressor_names}")
                    break

            if len(chosen_regressor_names) == 0:
                raise Exception("No regressors found.")

        #if spm_model.get("outlier"):
        #    
        #    outlier_indices = list(range(spm_model['outlier'].get("dummy_scan", 0)))
        #    
        #    if spm_model["outlier"].get("regressor_names"):
        #        
        #        outlier_names = ensure_list(spm_model["outlier"]["regressor_names"])
        #        
        #        col_exists = regressor_df.columns.isin(outlier_names)
        #        if col_exists.sum() > 0:
        #            chosen_outlier_names = regressor_df.columns[col_exists].tolist()
        #            outlier_indices = outlier_indices + list(np.ravel(np.where((regressor_df[chosen_outlier_names] != 0).any(axis=1))))
        #        else:
        #            raise Exception("No outliers found.")

        #    outlier_indices = np.unique(np.array(outlier_indices, dtype=int))
        #    
        #    if run != -1:
        #        outlier_file = os.path.join(outlier_path, f"sub-{sub}_task-{task}_run-{run}_outliers.txt")
        #    else:
        #        outlier_file = os.path.join(outlier_path, f"sub-{sub}_task-{task}_outliers.txt")
        #        
        #    outlier_files.append(outlier_file)               
        #    np.savetxt(os.path.join(env['data_path'], outlier_file), outlier_indices, fmt="%d")
            
        event_path = ensure_exist(env['data_path'], spm_model['event_files'].format(**format_args))

        if spm_model.get("pmod"):
            event_df = pd.read_csv(os.path.join(env['data_path'], event_path), sep='\t')
            
            for trial_type, pmods in spm_model['pmod'].items():
                trial_df = event_df.query(f'trial_type == "{trial_type}"')
                
                for pmod in ensure_list(pmods):
                    if trial_df[pmod].isna().any():
                        issues.append(f"{event_path}: '{pmod}' has missing values for '{trial_type}' events. They will be replaced by mean.")                        
                    if trial_df[pmod].var() == 0:
                        issues.append(f"{event_path}: '{pmod}' has zero variance for '{trial_type}' events, and will cause error. Consider excluding this run.")
        
        if (spm_model.get("event_options") is None) and (spm_model.get("pmod_options") is None):
            event_files.append(event)
        else:
            event_files.append(customize_event(env, job_name, spm_model, event_path))
                
    job['SpecifySPMModel']['functional_runs'] = functional_runs
    job['SpecifySPMModel']['event_files'] = event_files
    job['SpecifySPMModel']['regressors'] = regressors
    job['SpecifySPMModel']['regressor_names'] = chosen_regressor_names
    
    if len(outlier_files) > 0:
        job['SpecifySPMModel']['outlier_files'] = outlier_files
    
    job['SpecifySPMModel']['time_repetition'] = job["Info"]["tr"]
    job['Level1Design']['interscan_interval'] = job["Info"]["tr"]
    
    for key in ["event_options", "pmod_options", "outlier"]:
        remove_key(key, job["SpecifySPMModel"])

    if job['EstimateContrast'].get('basic_contrasts', False):
        all_events = pd.concat([pd.read_csv(os.path.join(env['data_path'], x), sep='\t') for x in job['SpecifySPMModel']['event_files']], 
                               ignore_index=True)
        
        contrasts = [] 
                               
        for trial_type in all_events['trial_type'].unique():
            contrasts.append([trial_type, "T", [trial_type], [1]])
        
        for cond, pmods in spm_model.get('pmod',{}).items():
            for pmod in ensure_list(pmods):
                contrasts.append([f'{cond}x{pmod}^1', "T", [f'{cond}x{pmod}^1'], [1]])
                
        contrasts = contrasts + ensure_list(job['EstimateContrast'].get('contrasts',[]))
            
        job['EstimateContrast']['contrasts'] = contrasts
        del job['EstimateContrast']['basic_contrasts']
    
    job["Info"]["sub"] = sub

    for key in ["job_path"]:
        remove_key(key, job["Environment"])

    for key in ["sub_container", "exclude", "run", "tr"]:
        remove_key(key, job["Info"])
                
    job_output = os.path.join(job_path, f"sub-{sub}.json")
    with open(job_output, 'w') as f:
        json.dump(job, f)
        
    if len(issues) == 0:
        print("job created")
    else:
        print("issues found - \n\t" + "\n\t".join(issues))

BA224 : job created
BA227 : job created
BA228 : job created
BA229 : job created
BA234 : job created
BA235 : job created
BA237 : job created
BA238 : job created
BA239 : job created
BA240 : job created
BA242 : job created
BA244 : job created
BA247 : job created
BA250 : job created
BA251 : job created
BA252 : job created
BA253 : job created
BA255 : job created
BA257 : job created
BA258 : job created
BA261 : job created
BA262 : job created
BA263 : job created
BA265 : job created
BA266 : job created
BA267 : job created
BA269 : job created
BA272 : job created
BA273 : job created
BA275 : job created
BA276 : job created
BA277 : job created
BA278 : job created
BA245 : job created


In [73]:
slurm_path = os.path.join(env['data_path'], env['job_path'], job_name, 'slurm') 
os.makedirs(slurm_path, exist_ok=True)
try:
    os.chmod(slurm_path, 0o0777)
except:
    pass
    
os.makedirs(os.path.join(slurm_path, 'out'), exist_ok=True)
os.chmod(os.path.join(slurm_path, 'out'), 0o0777)

for sub in subs:
    
    slurm_header = []
    slurm_header.append("#!/bin/bash")
    slurm_header.append(f"#SBATCH --job-name=sub-{sub}.job")
    slurm_header.append(f"#SBATCH --output=out/sub-{sub}.job")
    slurm_header.append(f"#SBATCH --error=out/sub-{sub}.err")
    slurm_header.append("#SBATCH --time=5-00:00")
    slurm_header.append("")
    slurm_header.append("srun ")

    data_path = env['data_path']
    output_path = env['output_path']
    working_path = env['working_path']

    json_path = os.path.join(job_path, f"sub-{sub}.json")
    
    cmd = []
    cmd.append("singularity run --cleanenv")
    cmd.append(f"-B {script_path}:/worker.py")
    cmd.append(f"-B {data_path}:/data")
    cmd.append(f"-B {output_path}:/output")
    cmd.append(f"-B {working_path}:/working")
    cmd.append(f"-B {json_path}:/job.json")
    cmd.append(f"{singularity_path} python /worker.py /job.json")
    
    slurm_output = os.path.join(output_path, job_name, f"sub-{sub}")
    slurm_working = os.path.join(working_path, job_name, f"sub-{sub}")
    
    slurm_footer = []
    slurm_footer.append("")
    slurm_footer.append("")
    slurm_footer.append(f"chmod -R 775 {slurm_output}")
    slurm_footer.append(f"chmod -R 775 {slurm_working}")
    
    slurm_cmd = "\n".join(slurm_header) + " \\\n  ".join(cmd) + "\n".join(slurm_footer) 
    
    with open(os.path.join(slurm_path, f"sub-{sub}.job"), 'w') as f:
        f.write(slurm_cmd)    

In [74]:
print(f"To test drive one job (sub-{sub}), copy and paste the following lines in terminal:")
print("")

print(" \\\n  ".join(cmd))

To test drive one job (sub-BA245), copy and paste the following lines in terminal:

singularity run --cleanenv \
  -B /data00/projects/megameta/scripts/jupyter_megameta/cnlab_pipeline/cnlab/GLM/first_level_HY.py:/worker.py \
  -B /data00/projects/megameta/BA:/data \
  -B /data00/projects/megameta/BA/derivatives/nipype:/output \
  -B /data00/projects/megameta/BA/working/nipype:/working \
  -B /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/jobs/sub-BA245.json:/job.json \
  /data00/tools/singularity_images/neurodocker.sqsh python /worker.py /job.json


In [75]:
print("Alternatively, submit the whole batch by copying and pasting the following lines in terminal:")
print("")

print(f"cd {slurm_path}")
for sub in subs:
    print(f"sbatch -D {slurm_path} -c 8 sub-{sub}.job")

Alternatively, submit the whole batch by copying and pasting the following lines in terminal:

cd /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm
sbatch -D /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm -c 8 sub-BA224.job
sbatch -D /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm -c 8 sub-BA227.job
sbatch -D /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm -c 8 sub-BA228.job
sbatch -D /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm -c 8 sub-BA229.job
sbatch -D /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm -c 8 sub-BA234.job
sbatch -D /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm -c 8 sub-BA235.job
sbatch -D /data00/projects/megameta/BA/models/task-walkstatement_model-messageUpdateTest/slurm -c 8 sub-BA237.job
sbatch -D /data00/p